In [ ]:
import pandas as pd
import matplotlib.pyplot as plt  
import numpy as np  
from sklearn.cluster import KMeans
import seaborn as sns
import jdatetime
import plotly.express as px
from khayyam import JalaliDate
import plotly.express as px

import warnings
warnings.filterwarnings("ignore")

In [4]:
df= pd.read_csv(r"/home/ccp/Desktop/task1/Stores_Transactions .csv")

In [5]:
# Convert Shamsi to Gregorian
df["transaction_date"] = df["transaction_date"].apply(lambda x: jdatetime.datetime.strptime(x, "%Y-%m-%d").togregorian())
df["transaction_date"] = pd.to_datetime(df["transaction_date"])

In [6]:

#Recency (R)
df['transaction_date'] = pd.to_datetime(df['transaction_date'])  
max_date=df['transaction_date'].max()
R = df.groupby('user_id').agg({'transaction_date': 'max'})
R['transaction_date']=R['transaction_date'].apply(lambda x: (max_date - x).days + 1)

#Frequency (F)
F= df.groupby(["user_id"]).agg({"transaction_id":"nunique"})

#Monetary (M)
M = df.groupby('user_id').agg({'total_price': 'sum'})

#RFM 
RFM = pd.concat([R, F, M], axis=1)
RFM.columns=['R', 'F', 'M']
RFM.head(12)

,R,F,M
user_id,,,
1,1,310,2.251340e+09
2,1,315,2.795794e+09
3,1,338,2.713384e+09
4,1,322,1.687042e+09
5,1,301,2.273373e+09
6,1,304,2.115132e+09
7,1,311,2.452395e+09
8,1,313,2.323948e+09
9,1,326,1.834763e+09


### all values in the R column are the same 

In [33]:
# RFM['R_score'] = pd.qcut(RFM['R'], q=5, labels=[5, 4, 3, 2, 1])   
RFM['F_score'] = pd.qcut(RFM['F'], q=5, labels=[1, 2, 3, 4, 5])#Splits th column into 5 equal-sized bins
RFM['M_score'] = pd.qcut(RFM['M'], q=5, labels=[1, 2, 3, 4, 5])

# Combine scores into a single RFM score
RFM['RFM_Score'] =RFM['F_score'].astype(str) + RFM['M_score'].astype(str)

# Define customer segments based on RFM scores
def assign_segment(row):
    if row['RFM_Score'] in ['55', '54', '45', '55']:
        return 'Champions'
    elif row['RFM_Score'] in ['44', '45', '54', '44']:
        return 'Loyal Customers'
    elif row['RFM_Score'] in ['35', '43', '45', '55']:
        return 'Potential Loyalists'
    elif row['RFM_Score'] in ['33', '23', '22', '21']:
        return 'Hibernating'
    else:
        return 'Lost Customers'

RFM['Segment'] = RFM.apply(assign_segment, axis=1)
RFM


,R,F,M,F_score,M_score,RFM_Score,Segment
user_id,,,,,,,
1,1,310,2.251340e+09,2,2,22,Hibernating
2,1,315,2.795794e+09,3,5,35,Potential Loyalists
3,1,338,2.713384e+09,5,5,55,Champions
4,1,322,1.687042e+09,4,1,41,Lost Customers
5,1,301,2.273373e+09,1,2,12,Lost Customers
6,1,304,2.115132e+09,1,1,11,Lost Customers
7,1,311,2.452395e+09,2,4,24,Lost Customers
8,1,313,2.323948e+09,3,3,33,Hibernating
9,1,326,1.834763e+09,4,1,41,Lost Customers


In [45]:

def plot_fm_iplot(RFM):
    # Define segment colors
    segment_colors = {
        'Champions': 'blue',
        'Loyal Customers': 'green',
        'Potential Loyalists': 'yellow',
        'Hibernating': 'orange',
        'Lost Customers': 'red'
    }

    # Assign colors based on segment
    RFM['Color'] = RFM['Segment'].map(segment_colors).fillna('gray')

    RFM_reset = RFM.reset_index() 

    
    fig = px.scatter(RFM_reset, x='F', y='M', color='Segment', 
                     color_discrete_map=segment_colors,
                     title="Customer Segmentation Based on Frequency & Monetary Value",
                     labels={'F': 'Frequency (Number of Purchases)', 'M': 'Monetary Value (Total Spend)'},
                     hover_data={'Customer_ID': RFM_reset.index, 'F': True, 'M': True})  # Show Customer ID in hover data

    # Highlight top customers by adding annotations
    top_customers = RFM.nlargest(5, 'M')
    for idx, row in top_customers.iterrows():
        fig.add_annotation(
            x=row['F']+0.2, y=row['M']+5, text=str(idx-1), 
            showarrow=True, arrowhead=2, ax=20, ay=-20, font=dict(size=12, color='black')
        )

    # Add grid and update layout for better presentation
    fig.update_layout(
        xaxis=dict(showgrid=True, gridwidth=0.5, gridcolor='LightGrey'),
        yaxis=dict(showgrid=True, gridwidth=0.5, gridcolor='LightGrey'),
        showlegend=True,
        legend_title="Customer Segments",
        template="plotly_white"
    )

    # Display the plot
    fig.show()

# Call the function
plot_fm_iplot(RFM)
